In [1]:
import re

def tokenizador(text):
    text = text.lower()
    intab = "áéíóú"
    outtab = "aeiou"
    str = text
    trantab = str.maketrans(intab, outtab)
    normalizado = str.translate(trantab)
    normalizado = re.sub(r'[^a-z0-9 ]','', normalizado)
    tokens = normalizado.split(" ")
    if "" in tokens:
        while "" in tokens:
            tokens.remove("")
    return tokens

In [2]:
import math
def idf(terms):
    idf = {}
    for term in terms:
        idf[term] = math.log(terms[term]["cf"]/ terms[term]["df"])
    return idf

        


In [3]:
import math
def get_score(index,queriText, idf):
    score = {}
    queri = tokenizador(queriText)
    for q in queri:
        for path,terms in index.items():
            if q in terms:
                tf = terms[q] / len(terms)
                if path in score:
                    score[path] +=  tf*idf[q]
                else:
                    score[path] = tf*idf[q] 
    return score
        

In [4]:
def contadorTerms(terms, tokens):
    for token in tokens:
        if token in terms:
            terms[token] += 1
        elif len(token)>2 and len(token)<20:
            terms[token] = 1
    return terms


In [5]:
import nltk
from nltk.corpus import stopwords
def remove_sportsworld(terms):
    dic = {}
    stopsSp = set(stopwords.words('spanish'))
    stopsEn = set(stopwords.words('english'))
    for term in terms:
        if term not in stopsSp and term not in stopsEn:
            dic[term] = terms[term]
    return dic

In [6]:
import os
root_dir = "wiki-small/"


def enumerate_html_files(directory):
    document_id = 1
    index = {}
    terms = {}
    for root, _, files in os.walk(directory):
        for file in files:
            dic = {}
            tokens = {}
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as html_file:
                termsAux = {}
                content = html_file.read()
                tokens =tokenizador(content)
                dic = contadorTerms(dic,tokens)
                termsAux = contadorTerms(termsAux,tokens)
                dic = remove_sportsworld(dic)
                index[file_path] = dic
                for term, aux in termsAux.items():
                    if term in terms:
                        terms[term]["cf"] += aux
                        terms[term]["df"] += 1
                    else:
                        terms[term] = {"cf": aux, "df": 1}
    return index, terms

index,terms = enumerate_html_files(root_dir)

In [7]:
idf = idf(terms)
idf

{'doctype': 0.0,
 'html': 0.00906027118664161,
 'public': 0.16386668874231478,
 'w3cdtd': 0.0,
 'xhtml': 0.0,
 'transitionalen': 0.0,
 'xmllangen': 0.0,
 'langen': 0.0006617039116544694,
 'dirltr': 0.00264419258343047,
 'head': 0.7507008313403902,
 'meta': 0.6935607968126881,
 'contenttexthtml': 0.0,
 'charsetutf8': 0.0,
 'headlinks': 0.0,
 'removed': 0.04560945723416245,
 'link': 0.7362237885691963,
 'relshortcut': 0.0,
 'icon': 0.008896269546257652,
 'hrefmiscfaviconico': 0.0,
 'titleibn': 0.47000362924573563,
 'ishaq': 1.6094379124341003,
 'wikipedia': 1.4894266804869947,
 'the': 3.569236651937325,
 'free': 1.4418541492072212,
 'encyclopediatitle': 0.0,
 'style': 1.2029131664904373,
 'typetextcsscdata': 0.0,
 'import': 0.0031392012561900057,
 'skinsofflinemaincss': 0.0,
 'relstylesheet': 0.0,
 'typetextcss': 0.0,
 'mediaprint': 0.00016546703107163415,
 '55000style': 0.6931471805599453,
 'typetextcssimport': 1.0986122886681098,
 '6style': 0.0,
 'iescript': 0.0,
 'typetextjavascript':

In [8]:

queri = 'software'
score=get_score(index,queri,idf )
score


{'wiki-small/en/articles/i/p/v/IPv6_fdb7.html': 0.0017670078703189637,
 'wiki-small/en/articles/i/n/t/Interstellar_Flames_2_bbdd.html': 0.0038102376605958343,
 'wiki-small/en/articles/i/n/t/Intellivoice.html': 0.0012898469901627921,
 'wiki-small/en/articles/i/g/o/IGo_d87c.html': 0.0020936253250852903,
 'wiki-small/en/articles/h/i/m/Himakuntla.html': 0.002461564429246319,
 'wiki-small/en/articles/h/e/a/Heart_of_Darkness_(video_game)_fcc7.html': 0.001027347137825943,
 'wiki-small/en/articles/h/e/r/Heraldo_Filipino_f40a.html': 0.001813075714522357,
 'wiki-small/en/articles/h/e/r/Hercules_(corporation).html': 0.0026033299199358976,
 'wiki-small/en/articles/h/p/i/Hping.html': 0.007871810259225696,
 'wiki-small/en/articles/b/u/d/Dragon_Ball_Z~_Budokai_2_ded8.html': 0.0007545311300572935,
 "wiki-small/en/articles/b/a/r/Bart_Simpson's_Escape_from_Camp_Deadly_55a9.html": 0.002089226952553598,
 'wiki-small/en/articles/b/r/a/Brave_Fencer_Musashi_22c6.html': 0.0007733063992344579,
 'wiki-small/en/

In [9]:
sorted_terms = sorted(score.items(), key=lambda item: item[1], reverse=True)
sorted_terms

[('wiki-small/en/articles/j/f/i/JFin_1830.html', 0.04105618470064044),
 ('wiki-small/en/articles/r/o/x/ROX_Desktop_b4f8.html', 0.038673912255047724),
 ('wiki-small/en/articles/a/u/t/Autodesk_Developer_Network_cf4f.html',
  0.0312273825661876),
 ('wiki-small/en/articles/s/t/e/Stellarium.html', 0.030693581154799778),
 ('wiki-small/en/articles/k/a/o/KAOS_(software_development)_665f.html',
  0.01668577230562941),
 ('wiki-small/en/articles/m/i/k/MiKTeX_fd58.html', 0.010468126625426451),
 ('wiki-small/en/articles/p/v/s/Pvserver.html', 0.010011463383377646),
 ('wiki-small/en/articles/e/l/e/Electronic_data_capture.html',
  0.00992817333859081),
 ('wiki-small/en/articles/r/e/a/Reactor_(Havok)_5dbb.html',
  0.009151583092780793),
 ('wiki-small/en/articles/p/p/3/PP3_444f.html', 0.008333564492308767),
 ('wiki-small/en/articles/m/i/c/Microtuner.html', 0.008196198044633347),
 ('wiki-small/en/articles/h/p/i/Hping.html', 0.007871810259225696),
 ('wiki-small/en/articles/o/p/e/OpenOffice.org_Impress_fe3

In [21]:
rankTfIdf = [tupla[0] for tupla in sorted_terms]
rankTfIdf

['wiki-small/en/articles/j/f/i/JFin_1830.html',
 'wiki-small/en/articles/r/o/x/ROX_Desktop_b4f8.html',
 'wiki-small/en/articles/a/u/t/Autodesk_Developer_Network_cf4f.html',
 'wiki-small/en/articles/s/t/e/Stellarium.html',
 'wiki-small/en/articles/k/a/o/KAOS_(software_development)_665f.html',
 'wiki-small/en/articles/m/i/k/MiKTeX_fd58.html',
 'wiki-small/en/articles/p/v/s/Pvserver.html',
 'wiki-small/en/articles/e/l/e/Electronic_data_capture.html',
 'wiki-small/en/articles/r/e/a/Reactor_(Havok)_5dbb.html',
 'wiki-small/en/articles/p/p/3/PP3_444f.html',
 'wiki-small/en/articles/m/i/c/Microtuner.html',
 'wiki-small/en/articles/h/p/i/Hping.html',
 'wiki-small/en/articles/o/p/e/OpenOffice.org_Impress_fe3f.html',
 'wiki-small/en/articles/s/o/f/SoftRAM_b9df.html',
 'wiki-small/en/articles/c/d/r/CDRWIN_b7e8.html',
 'wiki-small/en/articles/l/i/n/LinuxSysInfo_6788.html',
 'wiki-small/en/articles/j/o/h/John_Bridges_c39b.html',
 'wiki-small/en/articles/k/u/r/Kure.html',
 'wiki-small/en/articles/d/

In [17]:
import pyterrier as pt

if not pt.started():
    pt.init()

indexer = pt.FilesIndexer("./index", verbose=True, overwrite=True, meta={"docno":20, "filename":512})
indexref =indexer.index(root_dir)
index = pt.IndexFactory.of(indexref)

In [18]:
import numpy as np
br =  pt.BatchRetrieve(index, num_results=50, wmodel="TF_IDF", metadata=["filename"])
results = br.search(queri)

In [19]:
tfidf = np.array(results["filename"])
tfidf


array(['wiki-small//en/articles/j/f/i/JFin_1830.html',
       'wiki-small//en/articles/s/t/e/Stellarium.html',
       'wiki-small//en/articles/a/u/t/Autodesk_Developer_Network_cf4f.html',
       'wiki-small//en/articles/r/o/x/ROX_Desktop_b4f8.html',
       'wiki-small//en/articles/k/a/o/KAOS_(software_development)_665f.html',
       'wiki-small//en/articles/c/d/r/CDRWIN_b7e8.html',
       'wiki-small//en/articles/m/i/c/Microtuner.html',
       'wiki-small//en/articles/p/p/3/PP3_444f.html',
       'wiki-small//en/articles/f/o/r/Fortress_(software).html',
       'wiki-small//en/articles/y/a/s/Yass_(software).html',
       'wiki-small//en/articles/s/c/r/Scramdisk.html',
       'wiki-small//en/articles/e/l/e/Electronic_data_capture.html',
       'wiki-small//en/articles/p/i/r/Piriform.html',
       'wiki-small//en/articles/m/i/k/MiKTeX_fd58.html',
       'wiki-small//en/articles/n/+/_/N+.html',
       'wiki-small//en/articles/s/o/f/SoftRAM_b9df.html',
       'wiki-small//en/articles/c/l/x/

In [20]:
from scipy.stats import spearmanr

spearmancoefficient, datos = spearmanr(tfidf[:5], rankTfIdf[:5])
print("Coeficiente de correlación de Spearman 5 documentos:", spearmancoefficient,)

spearmancoefficient, datos = spearmanr(tfidf[:10], rankTfIdf[:10])
print("Coeficiente de correlación de Spearman 10 documentos:", spearmancoefficient,)


Coeficiente de correlación de Spearman 5 documentos: 0.8999999999999998
Coeficiente de correlación de Spearman 10 documentos: 0.5030303030303029
